# RNA2Sig classifier

Here we aim to classify the presence or absence of signatures in a sample given gene expression and one-hot encoded cancer type. Important notes:

* Gene expression is taken from TCGA, normalized by mean-centering across the pan-gyn cohort.
* We test two sets of genes:
    * The top 1000 highly variable genes from the C6 gene set from MSigDB (`c6_top1000`)
    * The L1000 landmark genes (`l1000`)
* We filter signatures to those with <1000 zero values to ensure that each is well-represented across the pan-gyn cohort.
* We remove signatures marked as sequencing artefacts according to COSMIC.
* We keep signatures with unknown aetiology according to COSMIC.
* For all signatures with exposure > 0, we assign it to 1 (signature is present). Otherwise, we assign it to 0 (signature is absent).

# Importing modules

In [1]:
print("Importing modules...")

# Core libraries
import os
import json
import random

# Numerical & data manipulation
import numpy as np
import pandas as pd

# Machine learning
import torch
import shap
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    accuracy_score,
    f1_score,
    classification_report,
    multilabel_confusion_matrix,
    precision_score,
    recall_score
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("Setting seed...")
def set_seed(seed):
    random.seed(seed)  # python random seed
    np.random.seed(seed)  # numpy random seed

seed = 9
set_seed(seed)

Importing modules...


Matplotlib created a temporary cache directory at /tmp/matplotlib-038jb2_4 because the default path (/gpfs/home/yb2612/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Setting seed...


# Defining functions

## Train/test

In [2]:
def train_val_multilabel(model_name, X_train, y_train, kfold=True, n_folds=5, n_jobs=-1, params_dict=None, print_params=True, seed=9):
    print()
    print(f"Training MultiOutputClassifier (XGBClassifier) on {model_name}...")
    print()

    if params_dict is None:
        params_dict = {
            "n_estimators": 100,
            "random_state": seed,
            "tree_method": "hist",
            "eval_metric": "logloss",
            "device":"cuda"
        }

    base_model = xgb.XGBClassifier(**params_dict)
    multi_model = MultiOutputClassifier(base_model, n_jobs=n_jobs)

    if print_params:
        print(f"{model_name} parameters:")
        print(json.dumps(base_model.get_params(), indent=2))

    if kfold:
        print(f"...Using {n_folds}-fold CV for {model_name}...")

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        y_pred = cross_val_predict(multi_model, X_train, y_train, cv=kf, n_jobs=n_jobs)

        # Calculate macro F1-score for all labels
        f1_macro = f1_score(y_train, y_pred, average='macro')
        print("----------------------------")
        print(f"{model_name} Cross-Validation Macro F1: {f1_macro:.4f}")
        print("----------------------------")

        # Train on full data
        multi_model.fit(X_train, y_train)
        
        # Return both the full multi-model and the list of individual estimators
        individual_models = multi_model.estimators_
        
        return multi_model, individual_models, y_pred, f1_macro

    else:
        print(f"...No k-fold CV for {model_name}...")
        multi_model.fit(X_train, y_train)
        
        # Return both the full multi-model and the list of individual estimators
        individual_models = multi_model.estimators_
        
        return multi_model, individual_models

def test_multilabel(models_list, X_test, Y_test, model_name, device="cuda"):
    print(f"Testing {model_name} on {device}...")
    
    # Check if CUDA is available
    if device == 'cuda' and not torch.cuda.is_available():
        print("Warning: CUDA requested but not available. Falling back to CPU.")
        device = 'cpu'
    
    # Initialize prediction array
    n_samples = X_test.shape[0]
    n_outputs = len(models_list)
    y_pred = np.zeros((n_samples, n_outputs), dtype=int)
    
    if device == 'cuda' and torch.cuda.is_available():
        print(f"Using CUDA for prediction with {torch.cuda.device_count()} GPU(s).")
        
        # For each estimator in the models list
        for i, estimator in enumerate(models_list):
            # Get the XGBoost booster
            booster = estimator.get_booster()
            
            # Create DMatrix for this prediction
            if isinstance(X_test, pd.DataFrame):
                feature_names = X_test.columns.tolist()
                X_test_dmatrix = xgb.DMatrix(X_test, feature_names=feature_names)
            else:
                X_test_dmatrix = xgb.DMatrix(X_test)
            
            # Make prediction - don't set predictor parameter here
            pred_proba = booster.predict(X_test_dmatrix)
            y_pred[:, i] = (pred_proba > 0.5).astype(int)
    else:
        # For CPU prediction, we need to construct predictions manually
        for i, estimator in enumerate(models_list):
            pred_proba = estimator.predict_proba(X_test)
            y_pred[:, i] = (pred_proba[:, 1] > 0.5).astype(int)
    
    # Calculate metrics
    f1_macro_test = f1_score(Y_test, y_pred, average='macro')
    print(f"Test Macro F1 score: {f1_macro_test:.4f}")
    
    return y_pred

## Plots

In [3]:
# Create the output directory if it doesn't exist
output_dir = "/gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier"
os.makedirs(output_dir, exist_ok=True)

def plot_classification_metrics(Y_test, y_pred, class_names, model_name="xgboost"):
    """Plot and save classification metrics"""
    # Initialize empty list for rows
    rows = []
    for idx, label in enumerate(class_names):
        y_true_label = Y_test.iloc[:, idx]
        y_pred_label = y_pred[:, idx]
        rows.append({
            'label': label,
            'precision': precision_score(y_true_label, y_pred_label, zero_division=1),
            'recall': recall_score(y_true_label, y_pred_label, zero_division=1),
            'f1-score': f1_score(y_true_label, y_pred_label, zero_division=1),
        })

    # Create DataFrame
    report_df = pd.DataFrame(rows).set_index('label')

    # Add overall row
    report_df.loc['overall'] = {
        'precision': precision_score(Y_test, y_pred, average='weighted', zero_division=1),
        'recall': recall_score(Y_test, y_pred, average='weighted', zero_division=1),
        'f1-score': f1_score(Y_test, y_pred, average='weighted', zero_division=1),
    }

    # Save metrics to CSV
    csv_path = os.path.join(output_dir, f"{model_name}_classification_metrics.csv")
    report_df.to_csv(csv_path)
    print(f"Saved classification metrics to: {csv_path}")

    # Plot
    plt.figure(figsize=(12, 6))
    ax = report_df.plot(kind='bar', figsize=(12, 6), colormap='viridis', edgecolor='black')
    plt.title("Classification Metrics by Class (Including Overall)", fontsize=14)
    plt.ylabel("Score")
    plt.ylim(0, 1.05)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Metric', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    
    # Save plot
    plot_path = os.path.join(output_dir, f"{model_name}_classification_metrics.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved classification metrics plot to: {plot_path}")
    
    return report_df

def plot_per_class_confusion_matrices(Y_test, y_pred, class_names, model_name="xgboost"):
    """
    Plots and saves a non-normalized confusion matrix per class (signature) for multilabel classification.
    Outputs one plot per signature.
    """
    mcm = multilabel_confusion_matrix(Y_test, y_pred)
    
    # Create a DataFrame to store confusion matrix data
    cm_data = []

    for i, cm in enumerate(mcm):
        class_name = class_names[i].replace(' ', '_').replace('/', '_')
        
        # Store confusion matrix data
        cm_data.append({
            'class': class_names[i],
            'true_neg': cm[0, 0],
            'false_pos': cm[0, 1],
            'false_neg': cm[1, 0],
            'true_pos': cm[1, 1]
        })
        
        plt.figure(figsize=(4, 4))
        ax = sns.heatmap(
            cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Pred 0", "Pred 1"], yticklabels=["True 0", "True 1"]
        )
        ax.set_title(f"Confusion Matrix for {class_names[i]}")
        ax.set_xlabel("Predicted Label")
        ax.set_ylabel("True Label")
        plt.tight_layout()
        
        # Save plot
        plot_path = os.path.join(output_dir, f"{model_name}_confusion_matrix_{class_name}.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved confusion matrix for {class_names[i]} to: {plot_path}")
    
    # Save confusion matrix data to CSV
    cm_df = pd.DataFrame(cm_data)
    csv_path = os.path.join(output_dir, f"{model_name}_confusion_matrices.csv")
    cm_df.to_csv(csv_path, index=False)
    print(f"Saved confusion matrix data to: {csv_path}")
    
    return cm_df

def plot_save_shap_values(models_list, X_test, feature_names=None, class_names=None, 
                         max_display=10, sample_size=100, seed=9, model_name="xgboost"):
    """
    Generate and save SHAP plots for multiple models in a MultiOutputClassifier.
    
    Parameters:
    -----------
    models_list : list
        List of trained XGBoost models from MultiOutputClassifier
    X_test : DataFrame or ndarray
        Test data for SHAP value calculation
    feature_names : list, optional
        Names of the features
    class_names : list, optional
        Names of the target classes/labels
    max_display : int, optional
        Maximum number of features to display in summary plots
    sample_size : int, optional
        Number of samples to use for SHAP value calculation (for large datasets)
    seed : int, optional
        Random seed for reproducibility when sampling
    model_name : str, optional
        Name of the model for file naming
    """
    # Convert pandas DataFrame to numpy if needed
    if hasattr(X_test, 'values'):
        if feature_names is None:
            feature_names = X_test.columns.tolist()
        X_test_values = X_test.values
    else:
        X_test_values = X_test
        
    # If we have too many samples, use a subset
    if sample_size and X_test_values.shape[0] > sample_size:
        np.random.seed(seed)  # Using the provided seed parameter
        sample_idx = np.random.choice(X_test_values.shape[0], sample_size, replace=False)
        X_test_sample = X_test_values[sample_idx]
    else:
        X_test_sample = X_test_values
    
    # Define class names if not provided
    if class_names is None:
        class_names = [f"Class_{i}" for i in range(len(models_list))]
    
    # Create DataFrame to store SHAP values
    all_shap_values = pd.DataFrame()
    
    # Loop through each model (one per target class)
    for i, model in enumerate(models_list):
        class_name = class_names[i].replace(' ', '_').replace('/', '_')
        print(f"\nGenerating SHAP plots for {class_names[i]}...")
        
        try:
            # Initialize SHAP explainer
            explainer = shap.TreeExplainer(model)
            
            # Calculate SHAP values
            shap_values = explainer.shap_values(X_test_sample)
            
            # For XGBoost binary classifiers, we might get a list of arrays
            if isinstance(shap_values, list):
                # For binary classification, we take the second class (positive class)
                shap_values = shap_values[1] if len(shap_values) > 1 else shap_values[0]
            
            # Calculate mean absolute SHAP value for each feature
            mean_abs_shap = np.abs(shap_values).mean(axis=0)
            
            # Create DataFrame for this class
            shap_df = pd.DataFrame({
                'Feature': feature_names if feature_names else [f"Feature_{j}" for j in range(len(mean_abs_shap))],
                'Mean_Abs_SHAP': mean_abs_shap,
                'Class': class_names[i]
            })
            shap_df = shap_df.sort_values('Mean_Abs_SHAP', ascending=False)
            
            # Add to collection
            all_shap_values = pd.concat([all_shap_values, shap_df])
            
            # Plot SHAP summary
            plt.figure(figsize=(6, 4))
            shap.summary_plot(
                shap_values, 
                X_test_sample, 
                feature_names=feature_names,
                max_display=max_display,
                show=False
            )
            # Simplify x-axis title
            plt.xlabel("SHAP value", fontsize=10)
            plt.title(f"SHAP Feature Importance for {class_names[i]}", fontsize=12)
            plt.tight_layout()
            
            # Save plot
            plot_path = os.path.join(output_dir, f"{model_name}_shap_summary_{class_name}.png")
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Saved SHAP summary plot to: {plot_path}")
            
            # Plot SHAP bar summary (average absolute SHAP values)
            plt.figure(figsize=(6, 4))
            shap.summary_plot(
                shap_values, 
                X_test_sample, 
                feature_names=feature_names,
                plot_type="bar",
                max_display=max_display,
                show=False
            )
            # Simplify x-axis title
            plt.xlabel("mean(|SHAP value|)", fontsize=10)
            plt.title(f"SHAP Mean Impact for {class_names[i]}", fontsize=12)
            plt.tight_layout()
            
            # Save plot
            plot_path = os.path.join(output_dir, f"{model_name}_shap_bar_{class_name}.png")
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Saved SHAP bar plot to: {plot_path}")
            
        except Exception as e:
            print(f"Error generating SHAP plots for {class_names[i]}: {str(e)}")
    
    # Save all SHAP values to CSV
    csv_path = os.path.join(output_dir, f"{model_name}_shap_values.csv")
    all_shap_values.to_csv(csv_path, index=False)
    print(f"Saved SHAP values to: {csv_path}")
    
    return all_shap_values

def plot_save_xgb_feature_importance(models_list, feature_names=None, class_names=None, 
                                    top_n=10, figsize=(6, 4), importance_type='total_gain',
                                    model_name="xgboost"):
    """
    Plot and save feature importance for each model in the MultiOutputClassifier.
    
    Parameters:
    -----------
    models_list : list
        List of trained XGBoost models from MultiOutputClassifier
    feature_names : list, optional
        Names of the features
    class_names : list, optional
        Names of the target classes/labels
    top_n : int, optional
        Number of top features to display
    figsize : tuple, optional
        Size of the figure
    importance_type : str, optional
        Type of importance to plot ('total_gain', 'gain', 'weight', 'total_cover', 'cover')
    model_name : str, optional
        Name of the model for file naming
    """
    # Define class names if not provided
    if class_names is None:
        class_names = [f"Class_{i}" for i in range(len(models_list))]
    
    # Create DataFrame to store all importance scores
    all_importance = pd.DataFrame()
    
    # Loop through each model (one per target class)
    for i, model in enumerate(models_list):
        class_name = class_names[i].replace(' ', '_').replace('/', '_')
        
        # Get the booster from the model
        booster = model.get_booster()
        
        # Get feature importance based on specified type
        importance_scores = booster.get_score(importance_type=importance_type)
        
        # Convert to dataframe for easier handling
        importance_df = pd.DataFrame({
            'Feature': list(importance_scores.keys()),
            'Importance': list(importance_scores.values()),
            'Class': class_names[i]
        })
        
        # Add to collection
        all_importance = pd.concat([all_importance, importance_df])
        
        # Sort by importance (descending)
        plot_df = importance_df.sort_values('Importance', ascending=False).head(top_n).iloc[::-1]
        
        # Plot
        plt.figure(figsize=figsize)
        plt.title(f"Feature Importance for {class_names[i]}", fontsize=12)
        plt.barh(range(len(plot_df)), plot_df['Importance'], align='center')
        
        # Add feature names to y-axis
        plt.yticks(range(len(plot_df)), plot_df['Feature'])
        
        plt.xlabel(f'Importance ({importance_type})')
        plt.tight_layout()
        
        # Save plot
        plot_path = os.path.join(output_dir, f"{model_name}_xgb_importance_{class_name}.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved XGBoost feature importance plot to: {plot_path}")
    
    # Save all importance values to CSV
    csv_path = os.path.join(output_dir, f"{model_name}_xgb_feature_importance.csv")
    all_importance.to_csv(csv_path, index=False)
    print(f"Saved XGBoost feature importance to: {csv_path}")
    
    return all_importance

# Combined function to run all analyses
def analyze_and_save_model_results(models_list, X_test, Y_test, y_pred, 
                                  feature_names=None, class_names=None,
                                  top_n=10, sample_size=100, model_name="xgboost"):
    """
    Run and save all analyses for the model in one go.
    
    Parameters:
    -----------
    models_list : list
        List of trained XGBoost models from MultiOutputClassifier
    X_test : DataFrame or ndarray
        Test data
    Y_test : DataFrame or ndarray
        True labels
    y_pred : ndarray
        Predicted labels
    feature_names : list, optional
        Names of the features
    class_names : list, optional
        Names of the target classes/labels
    top_n : int, optional
        Number of top features to display
    sample_size : int, optional
        Number of samples to use for SHAP analysis
    model_name : str, optional
        Name of the model for file naming
    """
    print(f"\n==== Saving All Results to: {output_dir} ====\n")
    
    # Extract feature names if available
    if feature_names is None and hasattr(X_test, 'columns'):
        feature_names = X_test.columns.tolist()
    
    # Extract class names if available
    if class_names is None and hasattr(Y_test, 'columns'):
        class_names = Y_test.columns.tolist()
    elif class_names is None:
        class_names = [f"Class_{i}" for i in range(Y_test.shape[1])]
    
    # 1. Classification metrics
    print("\n==== Plotting Classification Metrics ====")
    metrics_df = plot_classification_metrics(Y_test, y_pred, class_names, model_name)
    
    # 2. Confusion matrices
    print("\n==== Plotting Confusion Matrices ====")
    cm_df = plot_per_class_confusion_matrices(Y_test, y_pred, class_names, model_name)
    
    # 3. XGBoost Feature Importance
    print("\n==== Plotting XGBoost Feature Importance ====")
    xgb_importance = plot_save_xgb_feature_importance(
        models_list, 
        feature_names, 
        class_names, 
        top_n=top_n, 
        model_name=model_name
    )
    
    # 4. SHAP Analysis
    print("\n==== Plotting SHAP Analysis ====")
    shap_values = plot_save_shap_values(
        models_list, 
        X_test, 
        feature_names, 
        class_names, 
        max_display=top_n,
        sample_size=sample_size,
        model_name=model_name
    )
    
    print(f"\n==== Analysis Complete! All results saved to: {output_dir} ====")
    
    return {
        'metrics': metrics_df,
        'confusion_matrices': cm_df,
        'xgb_importance': xgb_importance,
        'shap_values': shap_values
    }

# Prediction pipeline

## c6_top1000

In [4]:
import pandas as pd 

model_name = "c6_top1000"
X_train = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/c6_top1000/splits/pangyn_X_train.csv", index_col=0)
X_test = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/c6_top1000/splits/pangyn_X_test.csv", index_col=0)
Y_train = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/c6_top1000/splits/pangyn_Y_train.csv", index_col=0)
Y_test = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/c6_top1000/splits/pangyn_Y_test.csv", index_col=0)

Y_cols_drop = ['SBS36', 'SBS34', 'SBS10a', 'SBS35', 'SBS31', 'SBS28', 'SBS98', 'SBS10b', 'SBS97', 'SBS96', 'SBS8', 'SBS26', 'SBS51', 'SBS54', 'SBS60']

print(Y_train.shape)
print(Y_test.shape)

Y_train.drop(columns=Y_cols_drop, inplace=True)
Y_test.drop(columns=Y_cols_drop, inplace=True)

print(Y_train.shape)
print(Y_test.shape)

Y_train = (Y_train > 0).astype(int)
Y_test = (Y_test > 0).astype(int)

(1420, 28)
(356, 28)
(1420, 13)
(356, 13)


In [5]:
params_dict = {
    "n_estimators": 700,
    "max_depth": 5,
    "learning_rate": 0.05,
    "random_state": 9,
    "tree_method": "hist",
    "eval_metric": "logloss",
    "device":"cuda"
}

# Run the training function (without k-fold for now)
model, models_list = train_val_multilabel(
    model_name=model_name, 
    X_train=X_train, 
    y_train=Y_train, 
    kfold=False,  # No k-fold, training on full data
    params_dict=params_dict,
    print_params=True
)

# Use the individual models list for prediction to properly handle GPU prediction
y_pred = test_multilabel(models_list, X_test, Y_test, model_name, device="cuda")

# Get your feature names (column names from your training data)
feature_names = X_train.columns.tolist()

# Get your class names (the target variables you're predicting)
class_names = Y_train.columns.tolist()


Training MultiOutputClassifier (XGBClassifier) on c6_top1000...

c6_top1000 parameters:
{
  "objective": "binary:logistic",
  "base_score": null,
  "booster": null,
  "callbacks": null,
  "colsample_bylevel": null,
  "colsample_bynode": null,
  "colsample_bytree": null,
  "device": "cuda",
  "early_stopping_rounds": null,
  "enable_categorical": false,
  "eval_metric": "logloss",
  "feature_types": null,
  "gamma": null,
  "grow_policy": null,
  "importance_type": null,
  "interaction_constraints": null,
  "learning_rate": 0.05,
  "max_bin": null,
  "max_cat_threshold": null,
  "max_cat_to_onehot": null,
  "max_delta_step": null,
  "max_depth": 5,
  "max_leaves": null,
  "min_child_weight": null,
  "missing": NaN,
  "monotone_constraints": null,
  "multi_strategy": null,
  "n_estimators": 700,
  "n_jobs": null,
  "num_parallel_tree": null,
  "random_state": 9,
  "reg_alpha": null,
  "reg_lambda": null,
  "sampling_method": null,
  "scale_pos_weight": null,
  "subsample": null,
  "tree

In [6]:
# Run the comprehensive analysis and save everything
results = analyze_and_save_model_results(
    models_list=models_list,
    X_test=X_test,
    Y_test=Y_test,
    y_pred=y_pred,
    feature_names=feature_names,
    class_names=class_names,
    top_n=10,              # Display only top 10 features
    sample_size=100,       # Use 100 samples for SHAP analysis
    model_name=model_name  # Customize model name for filenames
)

print("All plots and data have been saved!")


==== Saving All Results to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier ====


==== Plotting Classification Metrics ====
Saved classification metrics to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_classification_metrics.csv
Saved classification metrics plot to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_classification_metrics.png

==== Plotting Confusion Matrices ====
Saved confusion matrix for SBS1 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_confusion_matrix_SBS1.png
Saved confusion matrix for SBS10d to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_confusion_matrix_SBS10d.png
Saved confusion matrix for SBS13 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_confusion_matrix_SBS13.png
Saved confusion matrix for SBS14 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/c6_top1000_confusion_matrix_SBS14.png
Saved confusion 

<Figure size 1200x600 with 0 Axes>

## l1000

In [7]:
import pandas as pd 

model_name = "l1000"
X_train = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/l1000/splits/pangyn_X_train.csv", index_col=0)
X_test = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/l1000/splits/pangyn_X_test.csv", index_col=0)
Y_train = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/l1000/splits/pangyn_Y_train.csv", index_col=0)
Y_test = pd.read_csv("/gpfs/data/courses/aio2025/yb2612/data/rna2sig/l1000/splits/pangyn_Y_test.csv", index_col=0)

Y_cols_drop = ['SBS36', 'SBS34', 'SBS10a', 'SBS35', 'SBS31', 'SBS28', 'SBS98', 'SBS10b', 'SBS97', 'SBS96', 'SBS8', 'SBS26', 'SBS51', 'SBS54', 'SBS60']

print(Y_train.shape)
print(Y_test.shape)

Y_train.drop(columns=Y_cols_drop, inplace=True)
Y_test.drop(columns=Y_cols_drop, inplace=True)

print(Y_train.shape)
print(Y_test.shape)

Y_train = (Y_train > 0).astype(int)
Y_test = (Y_test > 0).astype(int)

(1420, 28)
(356, 28)
(1420, 13)
(356, 13)


In [8]:
params_dict = {
    "n_estimators": 700,
    "max_depth": 5,
    "learning_rate": 0.05,
    "random_state": 9,
    "tree_method": "hist",
    "eval_metric": "logloss",
    "device":"cuda"
}

# Run the training function (without k-fold for now)
model, models_list = train_val_multilabel(
    model_name=model_name, 
    X_train=X_train, 
    y_train=Y_train, 
    kfold=False,  # No k-fold, training on full data
    params_dict=params_dict,
    print_params=True
)

# Use the individual models list for prediction to properly handle GPU prediction
y_pred = test_multilabel(models_list, X_test, Y_test, model_name, device="cuda")

# Get your feature names (column names from your training data)
feature_names = X_train.columns.tolist()

# Get your class names (the target variables you're predicting)
class_names = Y_train.columns.tolist()


Training MultiOutputClassifier (XGBClassifier) on l1000...

l1000 parameters:
{
  "objective": "binary:logistic",
  "base_score": null,
  "booster": null,
  "callbacks": null,
  "colsample_bylevel": null,
  "colsample_bynode": null,
  "colsample_bytree": null,
  "device": "cuda",
  "early_stopping_rounds": null,
  "enable_categorical": false,
  "eval_metric": "logloss",
  "feature_types": null,
  "gamma": null,
  "grow_policy": null,
  "importance_type": null,
  "interaction_constraints": null,
  "learning_rate": 0.05,
  "max_bin": null,
  "max_cat_threshold": null,
  "max_cat_to_onehot": null,
  "max_delta_step": null,
  "max_depth": 5,
  "max_leaves": null,
  "min_child_weight": null,
  "missing": NaN,
  "monotone_constraints": null,
  "multi_strategy": null,
  "n_estimators": 700,
  "n_jobs": null,
  "num_parallel_tree": null,
  "random_state": 9,
  "reg_alpha": null,
  "reg_lambda": null,
  "sampling_method": null,
  "scale_pos_weight": null,
  "subsample": null,
  "tree_method": 

In [9]:
# Run the comprehensive analysis and save everything
results = analyze_and_save_model_results(
    models_list=models_list,
    X_test=X_test,
    Y_test=Y_test,
    y_pred=y_pred,
    feature_names=feature_names,
    class_names=class_names,
    top_n=10,              # Display only top 10 features
    sample_size=100,       # Use 100 samples for SHAP analysis
    model_name=model_name  # Customize model name for filenames
)

print("All plots and data have been saved!")


==== Saving All Results to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier ====


==== Plotting Classification Metrics ====
Saved classification metrics to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_classification_metrics.csv
Saved classification metrics plot to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_classification_metrics.png

==== Plotting Confusion Matrices ====
Saved confusion matrix for SBS1 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_confusion_matrix_SBS1.png
Saved confusion matrix for SBS10d to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_confusion_matrix_SBS10d.png
Saved confusion matrix for SBS13 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_confusion_matrix_SBS13.png
Saved confusion matrix for SBS14 to: /gpfs/home/yb2612/aio2025/yb2612/results/rec2sig/classifier/l1000_confusion_matrix_SBS14.png
Saved confusion matrix for SBS15 to: /gpfs/hom

<Figure size 1200x600 with 0 Axes>